In [39]:
import numpy as np
from scipy import spatial,stats
import pandas as pd
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [40]:
window_items =  [256, 1, 257, 6, 10, 12, 205, 206, 207, 210, 84, 212, 213, 214, 216, 217, 220, 121, 233, 234, 235, 239, 241, 243, 244, 245, 246, 247, 248, 249, 251, 252, 253, 254, 255] 
target_items = [2, 3, 4, 5, 7, 8, 9, 11, 13, 17, 211, 215, 219, 236, 250]

In [41]:
import os,sys
my_path = os.getcwd()
print (my_path)
path = os.path.join(my_path, "../data/ratings.txt")
print (path)

/home/surya/surya_114cs0121/AttackProfileDetectionRS/filmtrust/User-Based-Recommendation
/home/surya/surya_114cs0121/AttackProfileDetectionRS/filmtrust/User-Based-Recommendation/../data/ratings.txt


In [42]:
train = pd.read_csv(path,sep=' ',header=None,names=['userId','itemId','rating'])

In [43]:
train.describe()

,userId,itemId,rating
count,35497.000000,35497.000000,35497.000000
mean,744.247824,265.524946,3.002803
std,433.199457,346.854769,0.918692
min,1.000000,1.000000,0.500000
25%,356.000000,12.000000,2.500000
50%,735.000000,216.000000,3.000000
75%,1130.000000,251.000000,4.000000
max,1508.000000,2071.000000,4.000000


### Actual Average Rating of each item 

In [44]:
calculated_Rating = {}
items = train['itemId'].unique()

#rating
R = train.groupby(['itemId'])['rating'].mean()
for i in items:
#     print (i)
    calculated_Rating[i] = R[i]
    
# print (calculated_Rating)

In [45]:
test = pd.DataFrame()
for t_i in target_items:
    temp_df = train[train['itemId'] == t_i]
    train =  train.drop(train[train['itemId'] == t_i].index)
    test = pd.concat([temp_df, test]).drop_duplicates(keep=False)

### testing code of divising test/target data
table = [['Heading1', 'Heading2'], [1 , 1],[1 , 2],[1 , 3],[1 , 4], [1,5],[2 , 1],[2 , 2],[2 , 3],[2 , 4],[3, 1],[3, 2],[3, 3],[3, 4]]  
headers = table.pop(0)  
df = pd.DataFrame(table, columns=headers)

In [46]:
dfs = {n: g for n, g in test.groupby(test.groupby('itemId').cumcount() % 10)} 
test = pd.concat([dfs[0],dfs[1]], ignore_index=False)
train = pd.concat([train,dfs[2],dfs[3],dfs[4],dfs[5]], ignore_index=False)
del_data = pd.concat([dfs[6],dfs[7],dfs[8],dfs[9]], ignore_index=False)

In [47]:
train.shape[0]

28784

In [48]:
test.shape[0] + train.shape[0]

31036

In [49]:
test.tail(5)

,userId,itemId,rating
33607,1439,2,3.0
34026,1455,2,2.0
34499,1468,2,3.0
34934,1486,2,4.0
35399,1503,2,3.5


In [50]:
train.tail(5)

,userId,itemId,rating
33486,1433,2,4.0
33774,1446,2,3.5
34200,1460,2,3.0
34642,1475,2,2.5
35098,1495,2,3.0


In [51]:
del_data.tail(5)

,userId,itemId,rating
33540,1437,2,3.0
33958,1453,2,3.5
34403,1465,2,3.5
34879,1483,2,3.0
35344,1501,2,3.5


### Finding error for target items in actual data

In [52]:
target_item_error_in_actual_data = dict((t_i,[0,0]) for t_i in target_items)
print (target_item_error_in_actual_data)

{2: [0, 0], 3: [0, 0], 4: [0, 0], 5: [0, 0], 7: [0, 0], 8: [0, 0], 9: [0, 0], 11: [0, 0], 13: [0, 0], 17: [0, 0], 211: [0, 0], 215: [0, 0], 219: [0, 0], 236: [0, 0], 250: [0, 0]}


In [53]:
t1=time.time()
for index,i in enumerate(test.itertuples(),1):
    k_optimal = 25

    predicted=0
    usr=int(i.userId)
    mov=int(i.itemId)
    actual=i.rating
        
    all_ratings_for_the_active_item=train[train['itemId']==mov]
    list_of_movies_current_user_has_rated=train[train['itemId']==usr]
    similarity=[]
    corres_rate=[]
    
    for ind2,u in enumerate(all_ratings_for_the_active_item.itertuples(),1): 
        c1=[]
        c2=[]
        c_user=int(u.userId)
#         c_item=int(u.itemId)
        list_curuser=train[train['userId']==c_user]
#         list_curitem=train[train['itemId']==c_item]
        xt=pd.merge( list_of_movies_current_user_has_rated,list_curuser,how='inner',on=['itemId'])
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
        sim=0
        if len(c1_list)!=0:
            sim= stats.pearsonr(c1_list,c2_list)[0]#(1-spatial.distance.cosine(c1_list,c2_list))
        
        similarity.append(sim)
        corres_rate.append(u.rating)
        
    corres_rate = [x for _,x in sorted(zip(similarity,corres_rate),reverse=True)]
    similarity.sort(reverse=True)
#     print (mov)
#     print (similarity)
#     print (corres_rate)
    cs=0
    numc=0
    
    if(len(similarity)<k_optimal):
        k_optimal = len(similarity) 
        
    for k in range(k_optimal+1):        
#         print (k)
        cs+=similarity[k-1]
        numc+=(similarity[k-1]*corres_rate[k-1])
       
    if cs!=0:
        predicted=numc/cs
    else:
        predicted = 0
    
    if mov in target_item_error_in_actual_data:
        target_item_error_in_actual_data[mov][0] = target_item_error_in_actual_data[mov][0] + abs(predicted - calculated_Rating[mov])
        target_item_error_in_actual_data[mov][1] = target_item_error_in_actual_data[mov][1] +1
t2=time.time()        

/home/surya/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3003: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


KeyboardInterrupt: 

In [ ]:
index

In [ ]:
print (t2-t1)

In [ ]:
target_item_error_in_actual_data

### MAE error

In [ ]:
for i in target_item_error_in_actual_data:
    print (i,'    :',target_item_error_in_actual_data[i][0]/target_item_error_in_actual_data[i][1])

## Nuke Attack Strength Check

### Injecting Attack Profiles

In [ ]:
nuke_rating = 0.5

In [ ]:
attack_ids = list(range(1600,1700))

for attacker in attack_ids:
    for item in window_items:
        a = calculated_Rating[item]
        attack_rating = int(a) if (a-int(a)<0.5) else int(a)+1
        train = train.append(pd.Series([attacker,item,attack_rating], index=['userId', 'itemId' ,'rating']), ignore_index=True)

for attacker in attack_ids:
    for item in target_items:
        train = train.append(pd.Series([attacker,item,nuke_rating], index=['userId', 'itemId' ,'rating']), ignore_index=True)

In [ ]:
train.shape[0]

In [ ]:
train.tail(5)

## Finding error for target items in attack profile injected data

In [ ]:
target_item_error_in_actual_data = dict((t_i,[0,0]) for t_i in target_items)
print (target_item_error_in_actual_data)

In [ ]:
t1=time.time()
for index,i in enumerate(test.itertuples(),1):
    k_optimal = 25

    predicted=0
    usr=int(i.userId)
    mov=int(i.itemId)
    actual=i.rating
        
    all_ratings_for_the_active_item=train[train['itemId']==mov]
    list_of_movies_current_user_has_rated=train[train['itemId']==usr]
    similarity=[]
    corres_rate=[]
    
    for ind2,u in enumerate(all_ratings_for_the_active_item.itertuples(),1): 
        c1=[]
        c2=[]
        c_user=int(u.userId)
#         c_item=int(u.itemId)
        list_curuser=train[train['userId']==c_user]
#         list_curitem=train[train['itemId']==c_item]
        xt=pd.merge( list_of_movies_current_user_has_rated,list_curuser,how='inner',on=['itemId'])
        c1=xt['rating_x']
        c2=xt['rating_y']
        c1_list=c1.tolist()
        c2_list=c2.tolist()
        sim=0
        if len(c1_list)!=0:
            sim= stats.pearsonr(c1_list,c2_list)[0]#(1-spatial.distance.cosine(c1_list,c2_list))
        
        similarity.append(sim)
        corres_rate.append(u.rating)
        
    corres_rate = [x for _,x in sorted(zip(similarity,corres_rate),reverse=True)]
    similarity.sort(reverse=True)
#     print (mov)
#     print (similarity)
#     print (corres_rate)
    cs=0
    numc=0
    
    if(len(similarity)<k_optimal):
        k_optimal = len(similarity) 
        
    for k in range(k_optimal+1):        
#         print (k)
        cs+=similarity[k-1]
        numc+=(similarity[k-1]*corres_rate[k-1])
       
    if cs!=0:
        predicted=numc/cs
    else:
        predicted = 0
    
    if mov in target_item_error_in_actual_data:
        target_item_error_in_actual_data[mov][0] = target_item_error_in_actual_data[mov][0] + abs(predicted - calculated_Rating[mov])
        target_item_error_in_actual_data[mov][1] = target_item_error_in_actual_data[mov][1] +1
t2=time.time()        

In [ ]:
print (t2-t1)

In [ ]:
for i in target_item_error_in_actual_data:
    print (i,'    :',target_item_error_in_actual_data[i][0]/target_item_error_in_actual_data[i][1])

In [ ]:
2     : 1.449297167098073
3     : 1.6649957028852056
4     : 2.0105130428499165
5     : 1.298022430002044
7     : 1.2098419540229888
8     : 1.9130117838136704
9     : 1.5291516378358483
11     : 2.3077207822187047
13     : 1.6691777826372183
17     : 1.9058372428726094
211     : 1.4113157894736839
215     : 2.2161842285370126
219     : 2.291251830161054
236     : 1.3713275809869816
250     : 1.8801652892561982